In [1]:
import pandas as pd
from io import StringIO
import os

## Read trace

In [2]:
def read_trace(trace_dir, trace_file, worker_id=None):
    trace_filepath = os.path.join(trace_dir, trace_file)
    
    with open(trace_filepath,"r") as rf:
        new_trace = StringIO()
        line = rf.readline()
        while line:
            line_list = line.split()
            if line_list[0] == "[trace]":
                new_trace.write(line)
            line = rf.readline()
       
    new_trace.seek(0)
    df = pd.read_csv(new_trace, sep=" ", names=["InfoType", "EventType", "EventState", "ID", "Time"])
    del df["InfoType"]
    
    if worker_id is not None:
        df.insert(0, "WorkerID", worker_id)
        
    new_trace.close()
    
    return df

### COORD

In [3]:
nr_worker = 12
sub_dir = str(nr_worker) + "rm"
trace_dir = "/home/posheng/billy_ws/fl_025/fl_experiment_trace/nchc/" + sub_dir + "/mnist"
coord_trace_file = "coord.txt"

worker_files = []
for i in range(2, 2+nr_worker):
    worker_files.append("flvm-{}.txt".format(i))

# hpc0_trace_file = "hpc0.txt"
# hpc1_trace_file = "hpc1.txt"
# hpc2_trace_file = "hpc2.txt"
# hpc3_trace_file = "hpc3_trace.txt"

In [4]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

coord_df = read_trace(trace_dir, coord_trace_file)
# w0_df = read_trace(trace_dir, hpc0_trace_file, "alice")
# w1_df = read_trace(trace_dir, hpc1_trace_file, "bob")
# w2_df = read_trace(trace_dir, hpc2_trace_file, "charlie")

worker_dfs = {}
for i in range(len(worker_files)):
    worker_df = read_trace(trace_dir, worker_files[i], worker_files[i].split(".")[0])
    worker_dfs[worker_files[i].split(".")[0]] = worker_df

## Duration

In [5]:
def get_worker_duration(worker_id, worker_dfs, event_type):
    worker_df = worker_dfs[worker_id]
    worker_event_df = worker_df[worker_df["EventType"].str.contains(event_type, regex=False)]
    worker_event_df = worker_event_df.reset_index()
    
    del worker_event_df['index']
    del worker_event_df['ID']
    
    return worker_event_df

In [6]:
def get_coord_duration(coord_df, event_type):
    coord_event_df = coord_df[coord_df["EventType"].str.contains(event_type, regex=False)]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    del coord_event_df['ID']
    
    return coord_event_df

In [7]:
def calculate_duration(event_type, df):
    event_df = df[df["EventType"] == event_type][["EventType", "EventState", "ID", "Time"]]
    event_start_df = event_df[event_df["EventState"] == "start"][["ID", "Time"]]
    event_start_df = event_start_df.reset_index()

    event_end_df = event_df[event_df["EventState"] == "end"][["Time"]]
    event_end_df = event_end_df.reset_index()
    
    event_df_cat = pd.concat([event_start_df, event_end_df], axis=1)
    del event_df_cat['index']
    event_df_cat.columns = ["ID", "StartTime", "EndTime"]
    
    event_df_cat["Duration"] = event_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    event_df_cat.insert(0, "EventType", event_type)
    return event_df_cat


In [8]:
GlobalModelSendDuration = calculate_duration("GlobalModelSend", coord_df)

In [9]:
LossFuncSendDuration = calculate_duration("LossFuncSend", coord_df)

In [10]:
ModelConfigSendDuration = calculate_duration("ModelConfigSend", coord_df)

### GlobalModelSendDuration

In [11]:
for worker_id in worker_dfs.keys():
    gms_mean = GlobalModelSendDuration[GlobalModelSendDuration["ID"] == worker_id][1:]['Duration'].mean()
    print("GlobalModelSendDuration", worker_id, gms_mean)

GlobalModelSendDuration flvm-2 0.24629971981048585
GlobalModelSendDuration flvm-3 0.25056484937667844
GlobalModelSendDuration flvm-4 0.24529136419296266
GlobalModelSendDuration flvm-5 0.25251442193984985
GlobalModelSendDuration flvm-6 0.25657633543014524
GlobalModelSendDuration flvm-7 0.25854196548461916
GlobalModelSendDuration flvm-8 0.24981902837753295
GlobalModelSendDuration flvm-9 0.2549518346786499
GlobalModelSendDuration flvm-10 0.2536608338356018
GlobalModelSendDuration flvm-11 0.2572828531265259
GlobalModelSendDuration flvm-12 0.25408540964126586
GlobalModelSendDuration flvm-13 0.2504128456115723


### LossFuncSendDuration

In [12]:
for worker_id in worker_dfs.keys():
    lfs_mean = LossFuncSendDuration[LossFuncSendDuration["ID"] == worker_id][1:]['Duration'].mean()
    print("LossFuncSendDuration", worker_id, lfs_mean)

LossFuncSendDuration flvm-2 0.0013613104820251465
LossFuncSendDuration flvm-3 0.0012704730033874512
LossFuncSendDuration flvm-4 0.0012420296669006347
LossFuncSendDuration flvm-5 0.0014421701431274413
LossFuncSendDuration flvm-6 0.0014762163162231445
LossFuncSendDuration flvm-7 0.001211237907409668
LossFuncSendDuration flvm-8 0.0011418461799621582
LossFuncSendDuration flvm-9 0.0013916373252868652
LossFuncSendDuration flvm-10 0.0010529756546020508
LossFuncSendDuration flvm-11 0.0011195778846740724
LossFuncSendDuration flvm-12 0.0011324644088745116
LossFuncSendDuration flvm-13 0.0010709643363952636


### ModelConfigSendDuration

In [13]:
for worker_id in worker_dfs.keys():
    mcs_mean = ModelConfigSendDuration[ModelConfigSendDuration["ID"] == worker_id][1:]['Duration'].mean()
    print("ModelConfigSendDuration", worker_id, mcs_mean)

ModelConfigSendDuration flvm-2 0.0007476925849914551
ModelConfigSendDuration flvm-3 0.000826263427734375
ModelConfigSendDuration flvm-4 0.0007968306541442871
ModelConfigSendDuration flvm-5 0.0008943557739257812
ModelConfigSendDuration flvm-6 0.0007627010345458984
ModelConfigSendDuration flvm-7 0.0008275032043457032
ModelConfigSendDuration flvm-8 0.0008067727088928222
ModelConfigSendDuration flvm-9 0.0006778240203857422
ModelConfigSendDuration flvm-10 0.0006256461143493653
ModelConfigSendDuration flvm-11 0.0006460785865783691
ModelConfigSendDuration flvm-12 0.0007846593856811523
ModelConfigSendDuration flvm-13 0.0007852673530578613


## GlobalInformationSend Duration

In [14]:
def get_coord_global_send_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("GlobalInformationSend", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [15]:
for worker_id in worker_dfs.keys():
    gs_df = get_coord_global_send_duration(coord_df, worker_id)
    print("GlobalInformationSend", worker_id, gs_df['Time'][1:].mean())

GlobalInformationSend flvm-2 0.24548329114913942
GlobalInformationSend flvm-3 0.24831517934799194
GlobalInformationSend flvm-4 0.24452987909317017
GlobalInformationSend flvm-5 0.26234028339385984
GlobalInformationSend flvm-6 0.2594756603240967
GlobalInformationSend flvm-7 0.25748342275619507
GlobalInformationSend flvm-8 0.25332571268081666
GlobalInformationSend flvm-9 0.2485412359237671
GlobalInformationSend flvm-10 0.2567511796951294
GlobalInformationSend flvm-11 0.2628637790679932
GlobalInformationSend flvm-12 0.25974788665771487
GlobalInformationSend flvm-13 0.2555647850036621


## Send Time

In [16]:
## EventType:
##     * GlobalModelSend
##     * LossFuncSend
##     * ModelConfigSend
def calculate_send_time(coord_df, worker_dfs, event_type):
    event_start_df = coord_df[(coord_df["EventType"] == event_type) & (coord_df["EventState"] == "start")]
    worker_event_send_time_dfs = {}
    
    for worker_id in worker_dfs.keys():
        worker_event_start_df = event_start_df[event_start_df["ID"] == worker_id]
        worker_event_start_df = worker_event_start_df.reset_index()
        del worker_event_start_df['index']
        worker_event_start_df.columns = ["EventType", "EventState", "ID", "StartTime"]
        
        worker_df = worker_dfs[worker_id]
        worker_event_recv_df = worker_df[worker_df["EventType"] == event_type]
        worker_event_recv_df = worker_event_recv_df.reset_index()
        del worker_event_recv_df['index']
        worker_event_recv_df.columns = ["WorkerID", "EventType", "EventState", "ID", "RecvTime"]
        
        worker_recv_time_series = worker_event_recv_df["RecvTime"]
        worker_event_send_time_df = pd.concat([worker_event_start_df, worker_recv_time_series], axis=1)
        worker_event_send_time_df["SendTime"] = worker_event_send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
        del worker_event_send_time_df["EventState"]
        
        worker_event_send_time_dfs[worker_id] = worker_event_send_time_df
        
    return worker_event_send_time_dfs

In [17]:
# worker_gms_send_time_dfs = calculate_send_time(coord_df, worker_dfs, "GlobalModelSend")

In [18]:
# worker_gms_send_time_dfs['alice']['SendTime'].mean()

In [19]:
# worker_gms_send_time_dfs['alice']

In [20]:
# worker_gms_send_time_dfs['charlie']['SendTime'].mean()

In [21]:
# coord_df = read_trace(trace_dir, coord_trace_file)
# hpc1_df = read_trace(trace_dir, hpc1_trace_file, "alice")
# hpc2_df = read_trace(trace_dir, hpc2_trace_file, "bob")
# hpc3_df = read_trace(trace_dir, hpc3_trace_file, "charlie")

## LocalTraining Duration

In [22]:
for worker_id in worker_dfs.keys():
    lt_df = get_worker_duration(worker_id, worker_dfs, "LocalTraining")
    print("LocalTrainingDuration", worker_id, lt_df['Time'][1:].mean())

LocalTrainingDuration flvm-2 7.2970305442810055
LocalTrainingDuration flvm-3 9.358078587055207
LocalTrainingDuration flvm-4 6.418660485744477
LocalTrainingDuration flvm-5 7.354958999156952
LocalTrainingDuration flvm-6 8.44422767162323
LocalTrainingDuration flvm-7 5.800836300849914
LocalTrainingDuration flvm-8 7.598092746734619
LocalTrainingDuration flvm-9 8.45603688955307
LocalTrainingDuration flvm-10 6.092562031745911
LocalTrainingDuration flvm-11 7.206096565723419
LocalTrainingDuration flvm-12 9.471288347244263
LocalTrainingDuration flvm-13 6.922492897510528


## FitSagg Duration

In [23]:
def get_coord_fitsagg_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("FitSagg", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [24]:
for worker_id in worker_dfs.keys():
    fs_df = get_coord_fitsagg_duration(coord_df, worker_id)
    print("FitSaggDuration", worker_id, fs_df['Time'][1:].mean())

FitSaggDuration flvm-2 22.1480850815773
FitSaggDuration flvm-3 21.69628118276596
FitSaggDuration flvm-4 21.748558700084686
FitSaggDuration flvm-5 21.784575867652894
FitSaggDuration flvm-6 21.728962624073027
FitSaggDuration flvm-7 21.758643019199372
FitSaggDuration flvm-8 21.88845728635788
FitSaggDuration flvm-9 21.72841008901596
FitSaggDuration flvm-10 21.841166031360626
FitSaggDuration flvm-11 22.02138956785202
FitSaggDuration flvm-12 22.144763255119322
FitSaggDuration flvm-13 22.094984889030457


## EncryptMultiply Duration

In [26]:
for worker_id in worker_dfs.keys():
    em_df = get_worker_duration(worker_id, worker_dfs, "EncryptMultiply")
    print("EncryptMultiplyDuration", worker_id, em_df['Time'][1:].mean())

EncryptMultiplyDuration flvm-2 14.525941503047942
EncryptMultiplyDuration flvm-3 11.960908591747284
EncryptMultiplyDuration flvm-4 14.996360528469086
EncryptMultiplyDuration flvm-5 14.084626173973083
EncryptMultiplyDuration flvm-6 12.925717628002166
EncryptMultiplyDuration flvm-7 15.631257152557373
EncryptMultiplyDuration flvm-8 13.904936039447785
EncryptMultiplyDuration flvm-9 12.887753629684449
EncryptMultiplyDuration flvm-10 15.362527978420257
EncryptMultiplyDuration flvm-11 14.462584698200226
EncryptMultiplyDuration flvm-12 12.294977128505707
EncryptMultiplyDuration flvm-13 14.80768300294876


## EncryptParameter Duration

In [27]:
def get_encrypt_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('EncryptParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df
    

In [28]:
# # hpc1_ep_df = get_encrypt_parameter_duration("alice", worker_dfs)
# hpc1_ep_df = get_worker_duration("alice", worker_dfs, "EncryptParameter")
# hpc2_ep_df = get_worker_duration("bob", worker_dfs, "EncryptParameter")
# hpc3_ep_df = get_worker_duration("charlie", worker_dfs, "EncryptParameter")

In [29]:
# mean_values = []
# for i in range(8):
#     mean_value = hpc1_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
#     mean_values.append(mean_value)
#     print(str(i)+":", mean_value)

In [30]:
# sum(mean_values)

In [31]:
# for i in range(8):
#     mean_value = hpc2_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
#     print(str(i)+":", mean_value)

In [32]:
# for i in range(8):
#     mean_value = hpc3_ep_df[hpc1_ep_df["EventType"] == "EncryptParameter"+str(i)]["Time"].mean()
#     print(str(i)+":", mean_value)

## MultiplyParameter Duration

In [33]:
def get_multiply_parameter_duration(worker_id, worker_dfs):
    worker_df = worker_dfs[worker_id]
    worker_ep_df = worker_df[worker_df["EventType"].str.contains('MultiplyParameter', regex=False)]
    worker_ep_df = worker_ep_df.reset_index()
    
    del worker_ep_df["index"]
    del worker_ep_df["ID"]
    
    return worker_ep_df

In [34]:
# # hpc1_mp_df = get_multiply_parameter_duration("alice", worker_dfs)
# hpc1_mp_df = get_worker_duration("alice", worker_dfs, "MultiplyParameter")
# hpc2_mp_df = get_worker_duration("bob", worker_dfs, "MultiplyParameter")
# hpc3_mp_df = get_worker_duration("charlie", worker_dfs, "MultiplyParameter")

In [35]:
# hpc1_mp_df[:8]

In [36]:
# hpc1_mp_df[:8]["Time"].sum()

## RetrieveTime duration

In [37]:
def get_coord_retrieve_duration(coord_df, worker_id):
    coord_event_df = coord_df[coord_df["EventType"].str.contains("RetrieveTime", regex=False)]
    coord_event_df = coord_event_df[coord_event_df["ID"] == worker_id]
    coord_event_df = coord_event_df.reset_index()
    
    del coord_event_df['index']
    
    return coord_event_df

In [38]:
for worker_id in worker_dfs.keys():
    ret_df = get_coord_retrieve_duration(coord_df, worker_id)
    print("RetrieveTimeDuration", worker_id, ret_df["Time"][1:].mean())

RetrieveTimeDuration flvm-2 0.011370086669921875
RetrieveTimeDuration flvm-3 0.01518188714981079
RetrieveTimeDuration flvm-4 0.013814663887023926
RetrieveTimeDuration flvm-5 0.014360058307647704
RetrieveTimeDuration flvm-6 0.012369930744171143
RetrieveTimeDuration flvm-7 0.014160335063934326
RetrieveTimeDuration flvm-8 0.012493717670440673
RetrieveTimeDuration flvm-9 0.029947268962860107
RetrieveTimeDuration flvm-10 0.03087482452392578
RetrieveTimeDuration flvm-11 0.011914980411529542
RetrieveTimeDuration flvm-12 0.014117348194122314
RetrieveTimeDuration flvm-13 0.04642891883850098


## ShareSend Duration

In [39]:
def calculate_share_send_duration(worker_id, worker_dfs, nr_parameters):
    worker_df = worker_dfs[worker_id]
    worker_send_share_df = worker_df[worker_df["EventType"].str.contains('SendShare', regex=False)]
    
    worker_send_share_start_df = worker_send_share_df[worker_send_share_df["EventState"] == 'start']
    worker_send_share_start_df = worker_send_share_start_df.reset_index()
    worker_send_share_start_df = worker_send_share_start_df[["WorkerID", "EventType", "ID", "Time"]]
    worker_send_share_start_df.columns = ["WorkerID", "EventType", "ID", "StartTime"]
    
    worker_send_share_end_df = worker_send_share_df[worker_send_share_df["EventState"] == 'end']
    worker_send_share_end_df = worker_send_share_end_df.reset_index()
    
    send_share_df_cat = pd.concat([worker_send_share_start_df, worker_send_share_end_df["Time"]], axis=1)
    send_share_df_cat.columns = ["WorkerID", "EventType", "ID", "StartTime", "EndTime"]
    send_share_df_cat["Duration"] = send_share_df_cat.apply(lambda x: x["EndTime"] - x["StartTime"], axis=1)
    
    parameter_index = []
    nr_worker = len(worker_dfs)
    p_index = -1

    for i in range(len(send_share_df_cat)):
        tmp_index = (i%nr_worker)
        if tmp_index == 0:
            p_index += 1
            p_index = p_index % nr_parameters
        parameter_index.append(p_index)
        
    send_share_df_cat.insert(2, "p_idx", parameter_index)
    obj_id_list = send_share_df_cat.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_share_df_cat.insert(2, "obj_id", obj_id_list)
    
    del send_share_df_cat["EventType"]
    send_share_df_cat.insert(1, "EventType", "SendShare")
    
    return send_share_df_cat


In [40]:
# alice_send_share_df_cat = calculate_share_send_duration("alice", worker_dfs, 8)
# bob_send_share_df_cat = calculate_share_send_duration("bob", worker_dfs, 8)
# charlie_send_share_df_cat = calculate_share_send_duration("charlie", worker_dfs, 8)

## ShareSend Send Time

In [41]:
def calculate_share_send_time(sender_id, receiver_id, worker_dfs, nr_parameter):
    sender_df = worker_dfs[sender_id]
    receiver_df = worker_dfs[receiver_id]
    
    sender_send_share_df = sender_df[sender_df["EventType"].str.contains('SendShare', regex=False)]
    receiver_get_share_df = receiver_df[receiver_df["EventType"].str.contains('GetShare', regex=False)]
    
    send_share_start_df = sender_send_share_df[(sender_send_share_df["ID"] == receiver_id) & (sender_send_share_df["EventState"] == 'start')]
    get_share_df = receiver_get_share_df[receiver_get_share_df["ID"] == sender_id]
    send_share_start_df = send_share_start_df.reset_index()
    get_share_df = get_share_df.reset_index()
    
    del send_share_start_df['index']
    del send_share_start_df['EventState']
    
    send_time_df = pd.concat([send_share_start_df, get_share_df["Time"]], axis=1)
    send_time_df.columns = ["WorkerID", "EventType", "ID", "StartTime", "RecvTime"]
    send_time_df["SendTime"] = send_time_df.apply(lambda x: x["RecvTime"] - x["StartTime"], axis=1)
    
    nr_parameter = 8
    parameter_index = []

    for i in range(len(send_time_df)):
        tmp_index = (i%nr_parameter)
        parameter_index.append(tmp_index)
        
    send_time_df.insert(2, "p_idx", parameter_index)
    
    obj_id_list = send_time_df.apply(lambda x: x["EventType"].split("_")[1], axis=1)
    send_time_df.insert(2, "obj_id", obj_id_list)
    
    del send_time_df["EventType"]
    send_time_df.insert(1, "EventType", "SendShare")
    
    return send_time_df

In [42]:
# send_time_df = calculate_share_send_time("alice", "bob", worker_dfs, 8)

## AllWorkersTrainingTime Duration

In [43]:
coord_awtt_df = get_coord_duration(coord_df, "AllWorkersTrainingTime")

In [44]:
coord_awtt_df['Time'][1:].mean()

22.249258148670197

In [45]:
coord_awtt_df

,EventType,EventState,Time
0,AllWorkersTrainingTime,duration,22.346575
1,AllWorkersTrainingTime,duration,21.525356
2,AllWorkersTrainingTime,duration,20.572899
3,AllWorkersTrainingTime,duration,22.773884
4,AllWorkersTrainingTime,duration,22.912983
5,AllWorkersTrainingTime,duration,21.939624
6,AllWorkersTrainingTime,duration,21.741649
7,AllWorkersTrainingTime,duration,22.493547
8,AllWorkersTrainingTime,duration,22.373742
9,AllWorkersTrainingTime,duration,22.001366


## AggregationTime

In [46]:
coord_aggt_df = get_coord_duration(coord_df, "AggregationTime")

In [47]:
coord_aggt_df["Time"][1:].mean()

1.9175887942314147

In [48]:
coord_aggt_df

,EventType,EventState,Time
0,AggregationTime,duration,1.888433
1,AggregationTime,duration,2.052640
2,AggregationTime,duration,1.919819
3,AggregationTime,duration,2.011430
4,AggregationTime,duration,1.841063
5,AggregationTime,duration,2.017970
6,AggregationTime,duration,1.969205
7,AggregationTime,duration,1.968130
8,AggregationTime,duration,1.896335
9,AggregationTime,duration,1.918465


## DecryptionTime

In [49]:
coord_decrypt_df = get_coord_duration(coord_df, "DecryptionTime")

In [50]:
coord_decrypt_df['Time'][1:].mean()

0.3619753956794739

In [51]:
coord_decrypt_df

,EventType,EventState,Time
0,DecryptionTime,duration,0.364583
1,DecryptionTime,duration,0.366611
2,DecryptionTime,duration,0.354712
3,DecryptionTime,duration,0.380493
4,DecryptionTime,duration,0.359309
5,DecryptionTime,duration,0.365365
6,DecryptionTime,duration,0.377740
7,DecryptionTime,duration,0.362204
8,DecryptionTime,duration,0.362259
9,DecryptionTime,duration,0.355664


## RoundTime

In [52]:
coord_round_time_df = get_coord_duration(coord_df, "RoundTime")

In [53]:
coord_round_time_df["Time"].mean()

25.242705980936687

In [53]:
coord_round_time_df

,EventType,EventState,Time
0,RoundTime,duration,15.261874
1,RoundTime,duration,15.257234
2,RoundTime,duration,15.425665
3,RoundTime,duration,15.299137
4,RoundTime,duration,15.608480
5,RoundTime,duration,15.205121
6,RoundTime,duration,16.361328
7,RoundTime,duration,15.100507
8,RoundTime,duration,15.772338
9,RoundTime,duration,15.867190
